In [5]:
import pandas as pd
URM_path = "data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                sep=",",
                                # header=None,
                                dtype={0: int, 1: int, 2: int},
                                engine='python')

URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]
import scipy.sparse as sps

mapped_id, original_id = pd.factorize(URM_all_dataframe["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(URM_all_dataframe["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

In [12]:
from Data_manager.UserUtils import *
import pandas as pd
import numpy as np
from Recommenders.NonPersonalizedRecommender import TopPop


# Load the CSV file
file_path = 'submission_h.csv'
df = pd.read_csv(file_path)

URM_all = getURM_all()
users_few_iterations = URM_all.sum(axis=1)  
users_few_iterations = np.array(users_few_iterations).squeeze()
users_few_iterations = np.where((users_few_iterations < 2) * (users_few_iterations > 0))[0]

top = TopPop(URM_all)
top.fit()

item_index_to_original_id = item_original_ID_to_index.reset_index().set_index(0).to_numpy().reshape(-1)

update_dict = {}
for user_id in users_few_iterations:
    recommendations = [item_index_to_original_id[index] for index in top.recommend(user_id, cutoff=10)]
    recommendations = " ".join(str(x) for x in recommendations)
    user_original = user_original_ID_to_index.reset_index().set_index(0).to_numpy().reshape(-1)[user_id]
    update_dict[user_original] = recommendations



In [13]:
update_dict

{25: '2 4 1 7 3 6 8 9 15 20',
 31: '2 4 1 7 3 6 8 9 15 20',
 53: '2 4 1 7 3 6 8 9 15 20',
 66: '2 4 1 7 3 6 8 9 15 20',
 80: '2 4 1 7 3 6 8 9 15 20',
 105: '2 4 1 7 3 6 8 9 15 20',
 116: '2 4 1 7 3 6 8 9 15 20',
 120: '2 4 1 7 3 6 8 9 15 20',
 125: '2 4 1 7 3 6 8 9 15 20',
 161: '2 4 1 7 3 6 8 9 15 20',
 170: '2 4 1 7 3 6 8 9 15 20',
 186: '2 4 1 7 3 6 8 9 15 20',
 190: '2 4 1 3 6 8 9 15 20 14',
 195: '2 4 1 7 3 6 8 9 15 20',
 209: '2 4 1 7 3 6 8 9 15 20',
 222: '2 4 1 7 3 6 8 9 15 20',
 228: '2 4 1 7 3 6 8 9 15 20',
 238: '2 4 1 7 3 6 8 9 15 20',
 296: '2 4 1 7 3 6 8 9 15 20',
 304: '2 4 1 7 3 6 8 9 15 20',
 313: '2 4 1 7 3 6 8 9 15 20',
 335: '2 4 1 7 3 6 8 9 15 20',
 341: '2 4 1 7 3 6 8 9 15 20',
 353: '2 4 1 7 3 6 8 9 15 20',
 359: '2 4 1 7 3 6 8 9 15 20',
 378: '2 4 7 3 6 8 9 15 20 14',
 452: '4 1 7 3 6 8 9 15 20 14',
 507: '2 4 1 7 3 6 8 9 15 20',
 531: '2 4 1 7 3 6 8 9 15 20',
 548: '2 4 1 7 3 6 8 9 15 20',
 559: '2 4 1 7 3 6 8 9 15 20',
 562: '2 4 1 7 3 6 8 9 15 20',
 589: '2 4

In [14]:
# Update the item_list for the specified userIDs
for user_id, new_item_list in update_dict.items():
    df.loc[df['user_id'] == user_id, 'item_list'] = new_item_list

# Save the updated dataframe back to a CSV file
df.to_csv('updated_submission.csv', index=False)